Goal: determine the threshold of what it takes to be a playoff team

want to know for each season, what did the teams that made the playoffs do
then we will combine the qualifications to be a playoff team for each season to determine what our definition of a "playoff team" is

1. Separate regular season games by season
2. pivot data so that each team is an observation
    - create 4 new rows (away team winner/home team winner/away team loser/home team loser) to make more sense
    - create 2 new rows for goals scored and goals against
    - create new dataset with totals (ex team = philly, count up all the away wins for philly and put into a col, count up all the home wins for philly and put in new column)
3. add column that denotes whether a team made the playoffs

Notes:
- most seasons have 1230 games bc 82 games per team w/30 teams
- 2017/18 has 1271 games bc 82 games per team w/31 teams
- not using 2012/13 bc missing too much data
- not using 2018/19 because something is messed up w home_rink_side_start

In [245]:
import pandas as pd

In [246]:
games = pd.read_csv("data_cleaned/regular_season_games")

In [247]:
# dropping old indexes
games.drop('Unnamed: 0', axis = 1, inplace = True)

In [248]:
# creating new cols with away/home winner/loser
games['away_wins'] = games.loc[games.outcome.isin(['away win OT', 'away win REG']), 'away_team_id']
games['home_wins'] = games.loc[games.outcome.isin(['home win OT', 'home win REG']), 'home_team_id']
games['away_losses'] = games.loc[games.outcome.isin(['home win OT', 'home win REG']), 'away_team_id']
games['home_losses'] = games.loc[games.outcome.isin(['away win OT', 'away win REG']), 'home_team_id']

In [249]:
# creating df for each szn    
games_07_08 = games[games['season'] == 20072008]
games_08_09 = games[games['season'] == 20082009]
games_09_10 = games[games['season'] == 20092010]
games_10_11 = games[games['season'] == 20102011]
games_11_12 = games[games['season'] == 20112012]
games_13_14 = games[games['season'] == 20132014]
games_14_15 = games[games['season'] == 20142015]
games_15_16 = games[games['season'] == 20152016]
games_16_17 = games[games['season'] == 20162017]
games_17_18 = games[games['season'] == 20172018]

In [250]:
# importing df with team names
teams = pd.read_csv('data_cleaned/team_names.csv')

In [251]:
# getting rid of unused col
teams.drop('Unnamed: 0', axis = 1, inplace = True)

In [252]:
# creating teams df for each season
teams_07_08 = teams.copy()
teams_08_09 = teams.copy()
teams_09_10 = teams.copy()
teams_10_11 = teams.copy()
teams_11_12 = teams.copy()
teams_13_14 = teams.copy()
teams_14_15 = teams.copy()
teams_15_16 = teams.copy()
teams_16_17 = teams.copy()
teams_17_18 = teams.copy()

In [253]:
# function to add counts to new dfs
list_of_conditions = ['away_wins', 'home_wins', 'away_losses', 'home_losses']
list_of_teams = teams.team_name.values

def get_counts(games_df, team_df):
    for condition in list_of_conditions:
        for team in list_of_teams: 
            team_df.loc[team_df.team_name.isin([team]), condition] = len(games_df[games_df[condition] == team])

In [254]:
# adding counts
get_counts(games_07_08, teams_07_08)
get_counts(games_08_09, teams_08_09)
get_counts(games_09_10, teams_09_10)
get_counts(games_10_11, teams_10_11)
get_counts(games_11_12, teams_11_12)
get_counts(games_13_14, teams_13_14)
get_counts(games_14_15, teams_14_15)
get_counts(games_15_16, teams_15_16)
get_counts(games_16_17, teams_16_17)
get_counts(games_17_18, teams_17_18)

In [255]:
# function to get number of goals scored by a team
def total_goals_scored(team, games_df):
    return games_df.loc[games_df.away_team_id.isin([team]), 'away_goals'].sum() + games_df.loc[games_df.home_team_id.isin([team]), 'home_goals'].sum()

In [256]:
# function to get number of goals against a team
def total_goals_against(team, games_df):
    return games_df.loc[games_df.away_team_id.isin([team]), 'home_goals'].sum() + games_df.loc[games_df.home_team_id.isin([team]), 'away_goals'].sum()

In [264]:
# function to add total goals and goals against to teams datasets
def get_goal_counts(games_df, team_df):
    for team in list_of_teams:
        team_df.loc[team_df.team_name.isin([team]), 'total_goals_scored'] = total_goals_scored(team, games_df)
        team_df.loc[team_df.team_name.isin([team]), 'total_goals_against'] = total_goals_against(team, games_df)

In [261]:
get_goal_counts(games_07_08, teams_07_08)
get_goal_counts(games_08_09, teams_08_09)
get_goal_counts(games_09_10, teams_09_10)
get_goal_counts(games_10_11, teams_10_11)
get_goal_counts(games_11_12, teams_11_12)
get_goal_counts(games_13_14, teams_13_14)
get_goal_counts(games_14_15, teams_14_15)
get_goal_counts(games_15_16, teams_15_16)
get_goal_counts(games_16_17, teams_16_17)
get_goal_counts(games_17_18, teams_17_18)

In [263]:
teams_08_09

,team_name,away_wins,home_wins,away_losses,home_losses,total_goals_scored,total_goals_against
0,New Jersey Devils,23.0,28.0,18.0,13.0,244.0,209.0
1,Philadelphia Flyers,20.0,24.0,21.0,17.0,264.0,238.0
2,Los Angeles Kings,16.0,18.0,25.0,23.0,207.0,234.0
3,Tampa Bay Lightning,12.0,12.0,29.0,29.0,210.0,279.0
4,Boston Bruins,24.0,29.0,17.0,12.0,274.0,196.0
5,New York Rangers,17.0,26.0,24.0,15.0,210.0,218.0
6,Pittsburgh Penguins,20.0,25.0,21.0,16.0,264.0,239.0
7,Detroit Red Wings,24.0,27.0,17.0,14.0,295.0,244.0
8,San Jose Sharks,21.0,32.0,20.0,9.0,257.0,204.0
9,Nashville Predators,16.0,24.0,25.0,17.0,213.0,233.0
